这个guide主要说明怎么样完成下面这几个部分：
* 怎样管理自己的TensorFlow程序(__tf.Graph__)和TensorFlow的运行(runtime)(__tf.Session__),而不是通过Estimators去管理他们。
* 怎样通过__tf.Session__ 运行TensorFlow的Operations。
* 怎么样在low level的环境下使用high level的成分(datasets, layers, feature_columns)。
* 怎么样不适用Estimator而创建一个自己的training循环。
当可能的情况下，推荐使用高级别的API。知道TensorFlow的核心部分有以下原因：
* 当你可以使用一个low level TensorFlow operations的时候实验和调试都会更加直观。
* 让你知道在使用高级别的API的时候模型是怎么样内在运行的

## Setup
在每个程序的开始之前，先运行下面的代码：

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

## Tensor Values
TensorFlow的核心数据单元是__tensor__ 。一个tensor可以包含一组任意维度任意形状的原始数据值(primitive value)的数组。tensor's __rank__ 是它的维度，它的__shape__ 是一个元组的整数表明这个数组在不同维度上的长度。下面是tensor值得示例：

TensorFlow使用numpy数组来表明tensor值(__values__).

In [ ]:
3. # a rank 0 tensor; a scalar with shape [],
[1., 2., 3.] # a rank 1 tensor; a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

## TensorFlow Core Walkthrough
TensorFlow Core programs有下面两个分开的部分组成：
1. 创建计算图(a __tf.Graph__)
2. 运行计算图(using a __tf.Session__)
### Graph
__computational graph__ 是一系列的TensorFlow operations被组装到一个graph当中，这个graph包含以下两种类型的队形：
* Operations(或者说"ops"):是graph中的nodes。Operations描述consume和produce tensors的计算。
* Tensors: 是graph的边(edges)。这些代表在graph中流动的值，大部分的TensorFlow函数都会返回__tf.Tensors__ 。

> __Important：__ tf.Tensors没有值，他们只是在计算graph中的句柄。(tf.Tensors do not have values, they are just handles to elements in the computation graph.)

以下例子创建一个简单的计算图形。最基本的operation是常熟。 下面的Python函数创建了将tensor值作为输入的operations。这个操作的结果没有任何的输入，当运行的时候，这个输出的值会传递到构造器。我们可以创建两个浮点类型的常数a,b如下所示：

In [2]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


注意上面打印出来的tensor并没有输出如你所想的值3.0, 4.0, and 7.0。上面的成熟只会建立一个计算图。这些__tf.Tensor__对象值表示这些会运行的operations。

在图当中，每一个operation只会给一个唯一的名字。这个名字和Python当中分配给对象的名字是独立的。 Tensors的名字是创建他们的operation的名字的后面再加一个输出的索引号，如上面"add:0"所示。

### TensorBoard
TensorFlow提供一个叫做TensorBoard的工具包。TensorBoard能力中的一个就是显示计算图形。你可以通过下面简单的几句命令做到这点。

首先你需要保存一个计算图形到TensorBoard的总结文件，示例如下：

In [ ]:
writer = tf.summary.FileWrite('.')
writer.add_graph(tf.get_default_graph())

在上面的代码中会在现在的目录中创建一个有下面名字格式的__event__文件：
> events.out.tfevents.{timestamp}.{hostname}

现在，在一个新的终端，通过shell命令运行TensorBoard:
> tensorboard --logdir .

然后在浏览器中打开TensorBoard的[图形页面](http://localhost:6006/#graphs)， 你可以看到一个类似于下面的图形界面：
![Titile](getting_started_add.png)

更多的关于TensorBoard的图形显示的工具参加[ TensorBoard: Graph Visualization.](https://www.tensorflow.org/programmers_guide/graph_viz?hl=zh-cn)

### Session
为了评估tensors，实例化一个__tf.Session__对象，非正式的称呼为一个__session__。会话包含了TensorFlow的运行状态，还有TensorFlow operations的运行。 如果__tf.Graph__ 是一个像__.py__ 的文件，一个__tf.Session__ 是像python执行的。

下面的代码创建了一个__tf.Session__ 对象，并且唤醒了__run__ 方法来评估上面我们创建的__total__ tensor，示例如下1：

当你通过__Session.run__ 请求一个节点的输出。TensorFlow回溯到图形，计算所有对这个被请求输出节点提供输入的节点。所以这个被打印的值是7.0.

也可以传递多个tensors到 __tf.Session.run__ 。这个__run__ 方法透明的处理元组或者字典的组合，示例如下2：

这个返回值有一个相同得布局结构。

当调用__tf.Session.run__ 每个__tf.Tensor__ 只会有一个相同的值。例如，下面的代码中调用__tf.random_uniform__ 来长生一个__tf.Tensor__ 生成一个随机的 3-element矢量(只有值[0,1])，示例如3：

这个结果表明每次调用run会有不同的随机值，但是在同一个__run__当中会有相同的值（out1 and out2有相同的输入）

有一些TensorFlow函数返回 __tf.Operations__ 而不是__tf.Tensors__. 在Operation上面调用__run__的结果是None。你可以运行一个operation来导致一个副作用，而不是来取回一个值。详细查看下面初始化和训练的ops例子。

In [3]:
# 1：
sess = tf.Session()
print(sess.run(total))

# 2:
print (sess.run({'ab':(a, b), 'total':total}))

# 3:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

7.0
{'total': 7.0, 'ab': (3.0, 4.0)}
[ 0.53463995  0.49421775  0.71698451]
[ 0.65756512  0.53536665  0.03139257]
(array([ 1.81956518,  1.16343856,  1.00031149], dtype=float32), array([ 2.8195653 ,  2.16343856,  2.00031137], dtype=float32))


### Feeding
就目前看来(as it stand)，这个graph并不是特别有意义因为它只能永远产生一个常数值。一个graph可以被参数化接收外部输入，被称为__placeholders__。一个__placeholder__是稍后提供一个值得承诺，类似于函数得参数。

下面得开始得三行代码像是定义一个有两个输入参数(x,y)，还有一个操作作用于二者得函数。我们可以通过使用一个__feed_dict__参数到run方法来输入placeholder当中具体得值来评估这个graph。

如上说明，__feed_dict__参数是用来重写在graph中得任何tensor。placeholder和tf.Tensors得唯一区别就是如果说placeholder没有输入值得化会抛出一个错误。


In [4]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x+y
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

7.5
[ 3.  7.]


## Datasets
placeholder作用于简单得实现，但是Datasets是将数据流入模型当中更受喜爱得方法。

从Dataset当中获得一个可以运行的 tf.Tensor, 首先你必须将他转换成一个 __tf.data.Iterator__, 然后调用Iterator的 __get_next__ 方法。

最简单的创建一个Iterator的方法是用__make_one_shot_iterator__方法。 示例如下， __next_item__ tensor会在每次调用__run__的时候从__my_data__数组中返回一行。

当到达数据流的尾部，Dataset会抛出一个OutOfRangeError异常，一般用try except来捕获异常，如下所示：

In [5]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 3]
[4 5]
[6 7]


## Layers
一个可以训练的模型必须要调整图的值来用同样的输入获得不同的输出。 Layers是将trainable的参数添加到图中比较喜欢的方式。

Layers包中变量(variables)和操作(operations)都可以作用于之上。例如一个densely-connected layer将所有的输入通过一个权重加起来到每一个输出，再通过一个可以选择的激活函数。这些连接的权重和编制都是layer对象管理的。

### Creating Layers
下面的代码创建了一个Dense layer，输入时批量的输入矢量，每个都输出一个单一值。实行一个layer到输入，调用这个layer就像它时一个函数一样，示例如下：

In [7]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

这个layer 探查它的输入来决定它内部参数的个数（size）。因此这里我们必须设定x占位符的形状，这样layer可以创建一个正确大小的权重矩阵。

现在我们定义了计算的输出，y。这里还有在运行计算之前需要小新的消息信息。

### Initializing Layers
包含变量的layers必须要在使用之前初始化。虽然可以分别初始化变量，但是我们可以一次性初始化所有的变量通过下面的命令。

In [9]:
init = tf.global_variables_initializer()
sess.run(init)

> __important:__ 调用 tf.global_variables_initializer() 只会创建和返回一个句柄到TensorFlow operation。 这个op只有在运行__tf.Session.run__的时候才会初始化所有的变量。

如注意所示，global_variables_initializer只会初始化在initializer被创建的时候在图中存在的变量。所以，initializer必须时调价到graph构造中的最后一件事情。

### Executing Layers
当layer被初始化，可以评估linear——model的输出tensor，如同所有的其他tensor。例如，下面的代码产生一个二维的输出矢量：

In [10]:
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))

[[ 1.2563138 ]
 [ 1.60657215]]


### Layer Function shortcuts
对每个layer类(类似于tf.layer.Dense)。TensorFlow提供一个快捷功能函数。唯一的区别就时这个函数创建和运行通过一次调用。例如，下面的代码实现上面的功能。

虽然说便利，这个方法不允许进入(allow no access to) tf.layers.Layer 对象。这就让自检查和调试变得很困难，layer的重复使用也不可能。

In [11]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[ 1.83815849]
 [ 2.98474169]]


## Feature columns
用来使用feature columns的最简单的方式是使用__tf.feature_column.input_layer__ 函数。 这个函数接受__dense column__ 的输入，你必须包装它到_tf.feature_column.indicator_column__ 来展示一个categrical column。例如：

In [12]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

运行inputs tensor会解释features到batch of vectors。

Feature columns可以有内在的状态，例如layers，所以他们经常需要被初始化。categorical columns利用内在查阅表，这些都要求一个分开的初始化的op，__tf.tables_initializer__。


In [13]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

(None, None)

一旦内在状态被初始化，你可以运行inputs就像其他任何的tf.Tensor。

这些可展示了怎么样包装(packed)输入矢量，用one-hot "department"作为前两个索引，“sales"作为第三个。

In [14]:
print(sess.run(inputs))

[[  1.   0.   5.]
 [  1.   0.  10.]
 [  0.   1.   8.]
 [  0.   1.   9.]]


## Training
上面是TensorFlow的核心。现在手动训练一个小的回归模型。

### Define the data
首先需要定义一些输入x还有每个输入预期的输出y_true:

In [15]:
x = tf.constant([[1],[2],[3],[4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

### Define the model
然后，建立一个有一个输出的简单线性模型：  
在通过运行Session评估预测  
现在这个模型还没有被训练，所以者个预测的模型还不是很好。这里是我们获得的，每个人的输出都可能是不一样的：  

In [16]:
linear_model = tf.layers.Dense(units=1)
y_pred = linear_model(x)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[-1.72980392]
 [-3.45960784]
 [-5.18941164]
 [-6.91921568]]


#### loss
为了优化模型，你首先需要定义loss。在这里用MSE，回归问题的标准loss。

虽然你可以手动的通过数学操作来解决这个问题，__tf.losses__模块提供了一系列普通的loss函数。可以通过下面的代码来计算MSE。这个将产生一个loss值。

In [17]:
loss = tf.losses.mean_squared_error(labels=y_true,predictions=y_pred)
print(sess.run(loss))

8.64362


#### Training
TensorFlow提供了__optimizers__ 实现标准的优化算法。这些被作为__tf.train.Optimizer__ 的子类实现。他们累积的(incrementally)改变每个变量来减小loss。最简单的优化算法是梯度下降法(__gradient descent__）,通过__tf.train.GradientDescentOptimizer__ 实现。它根据变量的loss的微分的大小(magnitude)修改变量值。

这些代码创建优化必要的所有的graph成分，并且返回一个training operation。当运行的时候，这个training op会在graph中改变变量(update variables)。按照下面的代码可以运行，示例如下：

因为train是一个op而不是一个tensor，所以当运行的时候它不会返回一个值，为了在training的时候升级loss(progression of the loss)，可以在同一时间运行loss tensor，产生一个类似于下面的输出。

整体示例如下：

In [18]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

for i in range(100):
    _, loss_value = sess.run((train, loss))
    print(loss_value)

8.64362
6.02039
4.20004
2.93681
2.06014
1.45171
1.0294
0.736235
0.532682
0.391311
0.293086
0.224802
0.177292
0.144199
0.12111
0.104963
0.0936335
0.0856481
0.0799835
0.0759301
0.0729953
0.0708375
0.0692196
0.067977
0.0669955
0.0661959
0.0655233
0.0649394
0.0644178
0.0639402
0.0634937
0.0630696
0.0626616
0.0622655
0.0618783
0.061498
0.0611232
0.0607528
0.0603861
0.0600227
0.0596622
0.0593043
0.0589489
0.0585959
0.0582452
0.0578967
0.0575504
0.0572061
0.056864
0.056524
0.056186
0.05585
0.0555161
0.0551842
0.0548542
0.0545262
0.0542002
0.0538762
0.0535541
0.0532338
0.0529156
0.0525992
0.0522847
0.0519721
0.0516614
0.0513525
0.0510455
0.0507403
0.0504369
0.0501354
0.0498356
0.0495376
0.0492415
0.048947
0.0486544
0.0483635
0.0480744
0.0477869
0.0475012
0.0472172
0.0469349
0.0466543
0.0463754
0.0460981
0.0458225
0.0455485
0.0452762
0.0450055
0.0447364
0.0444689
0.0442031
0.0439388
0.0436761
0.043415
0.0431554
0.0428973
0.0426409
0.0423859
0.0421325
0.0418806


#### Complete program

In [19]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(100):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

print(sess.run(y_pred))

20.9443
14.6464
10.2757
7.24236
5.13688
3.67527
2.66043
1.95559
1.46587
1.12541
0.888519
0.723504
0.608365
0.527836
0.471327
0.431488
0.403221
0.382986
0.368328
0.357544
0.349451
0.34323
0.338311
0.334299
0.33092
0.327984
0.325358
0.322952
0.320701
0.318561
0.316503
0.314503
0.312549
0.310629
0.308736
0.306865
0.305013
0.303178
0.301357
0.299549
0.297754
0.295971
0.2942
0.29244
0.29069
0.288952
0.287224
0.285506
0.283799
0.282102
0.280415
0.278738
0.277072
0.275415
0.273769
0.272132
0.270505
0.268887
0.26728
0.265682
0.264093
0.262514
0.260945
0.259385
0.257834
0.256292
0.25476
0.253237
0.251723
0.250218
0.248722
0.247235
0.245756
0.244287
0.242826
0.241375
0.239932
0.238497
0.237071
0.235654
0.234245
0.232844
0.231452
0.230068
0.228693
0.227325
0.225966
0.224615
0.223272
0.221937
0.22061
0.219291
0.21798
0.216677
0.215382
0.214094
0.212814
0.211541
0.210277
0.209019
[[-0.73600465]
 [-1.35664451]
 [-1.97728443]
 [-2.59792423]]


Next steps
To learn more about building models with TensorFlow consider the following:

* [Custom Estimators](https://www.tensorflow.org/get_started/custom_estimators?hl=zh-cn), to learn how to build customized models with TensorFlow. Your knowledge of TensorFlow Core will help you understand and debug your own models.
If you want to learn more about the inner workings of TensorFlow consider the following documents, which go into more depth on many of the topics discussed here:

* [Graphs and Sessions](https://www.tensorflow.org/programmers_guide/graphs?hl=zh-cn)
* [Tensors](https://www.tensorflow.org/programmers_guide/tensors?hl=zh-cn)
* [Variables](https://www.tensorflow.org/programmers_guide/variables?hl=zh-cn)

In [20]:
224-11

213

In [21]:
213/4

53.25